In [56]:
## Stoichiometric Reactor, with recycle (and purge), material balance only
## Yicheng Hu, Victor M. Zavala, 2017
using JuMP
using Ipopt

# define algebraic model
m = Model(solver = IpoptSolver(tol = 0.01, max_iter = 15000));

# define sets 
K = ["NH3","H2","N2","CH4","AR"];         # set of components
F = ["ℓ1","ℓ2","ℓ3","ℓ4","ℓ5","ℓ6","ℓ7"]; # set streams

# define data
μf = [0; 30; 10; 0.001; 0.001]  # input flows [mol/s]
μf = Dict(zip(K,μf));           # assign to set          

# define variables
@variable(m,      μ[K,F]>=0)  # molar flows
@variable(m,    μ_tot[F]>=0)  # molar flows
@variable(m, 0 <= y[K,F] <= 1); # molar fractions 

# compute total flows and mol fractions
  @constraint(m, [ℓ in F],         μ_tot[ℓ] == sum(μ[k,ℓ] for k in K));
@NLconstraint(m, [k in K, ℓ in F],   y[k,ℓ] == μ[k,ℓ]/μ_tot[ℓ]);
            
# Block1: Mixer
@constraint(m, [k in K], μ[k,"ℓ1"] == μf[k]); 
@constraint(m, [k in K], μ[k,"ℓ2"] == μ[k,"ℓ1"] + μ[k,"ℓ7"]);

# Block2: Reactor
       γ = [2/3; -1; -1/3; 0; 0]; # stochiometric coefficients
       γ = Dict(zip(K,γ));        # assign to set          
    klim = "H2";                  # limiting component
@variable(m, η==0.3);             # extent of reaction 
            
@constraint(m, [k in K], μ[k,"ℓ3"] == μ[k,"ℓ2"] + η * (γ[k]/abs(γ[klim])) * μ[klim,"ℓ2"]);

# Block3: Flash separator
   ξf = [0.1;0.99;0.99;0.95;0.95];  # split fraction for each component
   ξf = Dict(zip(K,ξf));            # assign to set

@constraint(m, [k in K], μ[k,"ℓ3"] == μ[k,"ℓ4"] + μ[k,"ℓ5"]);
@constraint(m, [k in K], μ[k,"ℓ4"] == ξf[k]*μ[k,"ℓ3"]);

# Block4: Splitter
@variable(m, ξs==0.01);             # split fraction 
@constraint(m, [k in K], μ[k,"ℓ6"] == μ[k,"ℓ4"] * ξs);
@constraint(m, [k in K], μ[k,"ℓ7"] == μ[k,"ℓ4"]*(1-ξs));

solve(m)

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      232
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      105

Total number of variables............................:       77
                     variables with only lower bounds:       42
                variables with lower and upper bounds:       35
                     variables with only upper bounds:        0
Total number of equality constraints.................:       77
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

:Optimal

In [57]:
println(getvalue(μ))
println(getvalue(y))

μ: 2 dimensions:
[NH3,:]
  [NH3,ℓ1] = 1.62602452822116e-6
  [NH3,ℓ2] = 2.1000479725936394
  [NH3,ℓ3] = 21.212590105132687
  [NH3,ℓ4] = 2.1212589821663785
  [NH3,ℓ5] = 19.091331120714184
  [NH3,ℓ6] = 0.021212589822450997
  [NH3,ℓ7] = 2.1000463668187996
[ H2,:]
  [ H2,ℓ1] = 30.000000368306292
  [ H2,ℓ2] = 95.56271072603751
  [ H2,ℓ3] = 66.89389784231506
  [ H2,ℓ4] = 66.22495921420497
  [ H2,ℓ5] = 0.6689386269297093
  [ H2,ℓ6] = 0.662249591894051
  [ H2,ℓ7] = 65.56270998492394
[ N2,:]
  [ N2,ℓ1] = 9.999995747373246
  [ N2,ℓ2] = 31.853483173252577
  [ N2,ℓ3] = 22.29721162404442
  [ N2,ℓ4] = 22.074236832005862
  [ N2,ℓ5] = 0.2229747908944807
  [ N2,ℓ6] = 0.22074236662188765
  [ N2,ℓ7] = 21.85349060280382
[CH4,:]
  [CH4,ℓ1] = 0.0009996086050200205
  [CH4,ℓ2] = 0.016774065487546112
  [CH4,ℓ3] = 0.016773680882347885
  [CH4,ℓ4] = 0.01593459394937074
  [CH4,ℓ5] = 0.0008390899969098539
  [CH4,ℓ6] = 0.0001593680467771741
  [CH4,ℓ7] = 0.015774849648988582
[ AR,:]
  [ AR,ℓ1] = 0.0009996086050200205


In [58]:
println(getvalue(μ["NH3","ℓ5"]))
println(getvalue(y["NH3","ℓ5"]))
println(getvalue(μ_tot["ℓ7"]))

19.091331120714184
0.9552867123276326
89.54779664868956
